https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

nc -lk 9999 #initial

In [1]:
from pyspark import SparkContext

In [2]:
from pyspark.streaming import StreamingContext

In [3]:
from pyspark.sql import Row, SparkSession

In [4]:
from pyspark.sql.functions import regexp_extract

In [5]:
spark = SparkSession.builder\
.appName("StructuredStreaming")\
.getOrCreate()


22/07/07 09:59:51 WARN Utils: Your hostname, ubuntu1-virtual-machine resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/07/07 09:59:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/07 09:59:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/07 09:59:52 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 8c:23:21:0b:17:cf:8c:28


In [6]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

22/07/07 09:59:53 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [7]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [8]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [9]:
wordCounts = words.groupBy("word").count()

In [ ]:
query = wordCounts \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination()

22/07/07 10:00:04 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-641562c9-53ba-4205-b0a4-b3434e4bda0a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/07/07 10:00:04 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|  dd|    1|
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
| ria|    1|
+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|  dd|    2|
+----+-----+



In [ ]:
#query.stop()